In [2]:
using JuMP, Ipopt, Optim 

#Defining Sets
I = ["Soyabean","Textile"] #Soyabean is capital intensive and Textile is labour intensive in baseline 
J=["Labour","Capital"] # USA has less labor and more capital, CHINA has more labor and less capital
C=["USA","CHINA"];
#WELW=[0.5, 0.5]
len_C= size(C)[1]
len_I= size(I)[1]
len_J= size(J)[1];

# BASELINE: 

#parameters: these will change according to real values/dimensions 
BETA_IJ= [0.25 0.75; 0.75  0.25] #share of factor j in the production of good i
GAMMA_I= [0.5, 0.5]   #share of good i in the utility function
TC_IC=[1.000 1.000; 1.000 1.000]; #trade cost of importing good i for country c
VS_JC=[500 1500; 1500 500]  #endowment of factor j in country c

model1 = Model(Ipopt.Optimizer);

@variable(model1, TR_IC[1:len_I,1:len_C]);   #tariff on good I of country C 
@variable(model1, X_IC[1:len_I,1:len_C]>=0); # domestic production good i in country 
@variable(model1, M_IC[1:len_I,1:len_C]>=0); # import good i in country c
@variable(model1, E_IC[1:len_I,1:len_C]>=0); # export good i from country c
@variable(model1, U_C[1:1,1:len_C]); # Utility in country c
@variable(model1, PX_IC[1:len_I,1:len_C]>=0); # price of good i in country c
@variable(model1, PW_I[1:1,1:len_I]>=0); # world price of good i #changing
@variable(model1, PV_JC[1:len_J,1:len_C]>=0); # price of factor j in country c
@variable(model1, CONS_C[1:1,1:len_C]>=0); # consumption value in country c
@variable(model1, PU_C[1:1,1:len_C]>=0); # price of utility in country c

#zero profit conditions: Marginal Costs balance Marginal Revenue/Price
@NLconstraints model1 begin
#For domestic soyabean and textile 
    PRX[i = 1:len_I, c= 1:len_C], prod(PV_JC[j,c]^BETA_IJ[i,j] for j in 1:len_J) >= PX_IC[i,c]
#For imported soyabean and textile 
    PRM[i = 1:len_I, c=1:len_C], PW_I[i]*TC_IC[i,c]*TR_IC[i,c] >= PX_IC[i,c];
#For exported soyabean and textile 
    PRE[i = 1:len_I, c=1:len_C], PX_IC[i,c]>=PW_I[i]
#For utility 
    PRU[c=1:len_C], prod(PX_IC[i,c]^GAMMA_I[i] for i in 1:len_I)>=PU_C[c]
end;

#market clearing inequalities: Demands balance Supply
@NLconstraints model1 begin
#For domestic soyabean and textile   
    MKX[i = 1:len_I, c= 1:len_C], X_IC[i,c]- E_IC[i,c] + M_IC[i,c]/TC_IC[i,c] >= 
        prod( PX_IC[i,c]^GAMMA_I[i]  for i in 1:len_I)*(GAMMA_I[i]/PX_IC[i,c])*U_C[c] 
#For all imports and exports   
    MKW[i = 1:len_I], sum( E_IC[i,c] - M_IC[i,c] for c in 1:len_C) >=0 #chaning this to equality 
#For labor and capital inputs 
    MKV[j = 1:len_J, c= 1:len_C], VS_JC[j,c]>= sum( prod( PV_JC[j,c]^BETA_IJ[i,j] for j in 1:len_J)
            *(BETA_IJ[i,j]/PV_JC[j,c])*X_IC[i,c]   for i in 1:len_I)
#For utility
    MKU[c = 1:len_C], U_C[c]==CONS_C[c]/PU_C[c]
end; 

#Income balance equations: Incomes balance expenditures
@NLconstraints model1 begin
    ICONS[c = 1:len_C], CONS_C[c] == sum( VS_JC[j,c]*PV_JC[j,c] for j in 1:len_J) + sum( PW_I[i]*(TR_IC[i,c]-1)*M_IC[i,c] for i in 1:len_I)
end;

#numeraire: PU.FX("H") = 1
@constraint(model1, PU_C[1,1]==1 ) #relative prices 
@constraint(model1, TR_IC[1,1]==1 ) #Free Trade Condition
@constraint(model1, TR_IC[1,2]==1 ) 
@constraint(model1, TR_IC[2,1]==1 ) 
@constraint(model1, TR_IC[2,2]==1 ) 

#define objective
#@NLobjective(model1, Max, prod(U_C[i]^WELW[i] for i in 1:len_C )); #joint welfare maximization
@NLobjective(model1, Max, 0 )

# Solve the Model
@time optimize!(model1)

# Objective
exp1_obj=objective_value(model1) 



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       29
Number of nonzeros in inequality constraint Jacobian.:       86
Number of nonzeros in Lagrangian Hessian.............:      120

Total number of variables............................:       32
                     variables with only lower bounds:       26
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        9
Total number of inequality co

0.0

In [98]:
#print(model1)

In [3]:
exp1_TR_IC = value.(TR_IC) #tariff on good I of country C 
exp1_X_IC = value.(X_IC) # domestic production good i in country c 
exp1_M_IC = value.(M_IC) # import good i in country c
exp1_E_IC = value.(E_IC) # Export of good i by country c
exp1_U_C = value.(U_C) # Utility in country c
exp1_PX_IC = value.(PX_IC) # price of good i in country c
exp1_PW_I = value.(PW_I) # world price of good i
exp1_PV_JC = value.(PV_JC) # price of factor j in country c
exp1_CONS_C = value.(CONS_C) # consumption value in country c
exp1_PU_C = value.(PU_C) # price of utility in country c
exp1_U_C = value.(U_C) # price of utility in country c;
#Baseline Cons
exp1_CONS_C
#Baseline Utility
exp1_U_C
#Baseline Production: USA produces more soyabean and China produces more textiles
exp1_X_IC 
#Baseline Net Exports IC: USA exports soyabean and imports textiles; China exports textiles and imports soyabean
ES=exp1_E_IC.-exp1_M_IC
#Terms of Trade: equals 1 in baseline like all other prices
TOT_US= exp1_PW_I[1]/exp1_PW_I[2]
TOT_CHN=1/TOT_US

1.0000000000000029